# Library

In [11]:
import os
import zipfile
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data

In [12]:
source_path = '/Users/bened/Python/Farmku/Rice/RiceDiseaseDataset'
training_path = os.path.join(source_path, 'train')
validation_path = os.path.join(source_path, 'validation')

Split validation dataset into validation dataset and test dataset

In [13]:
# Create the testing dataset path
testing_path = os.path.join(source_path, 'test')

brownspot_testing_path = os.path.join(testing_path, 'BrownSpot')
healthy_testing_path = os.path.join(testing_path, 'Healthy')
hispa_testing_path = os.path.join(testing_path, 'Hispa')
leafblast_testing_path = os.path.join(testing_path, 'LeafBlast')

if not os.path.isdir(testing_path):
  os.makedirs(testing_path)
if not os.path.isdir(brownspot_testing_path):
  os.makedirs(brownspot_testing_path)
if not os.path.isdir(healthy_testing_path):
  os.makedirs(healthy_testing_path)
if not os.path.isdir(hispa_testing_path):
  os.makedirs(hispa_testing_path)
if not os.path.isdir(leafblast_testing_path):
  os.makedirs(leafblast_testing_path)

# Training
brownspot_training_path = os.path.join(training_path, 'BrownSpot')
healthy_training_path = os.path.join(training_path, 'Healthy')
hispa_training_path = os.path.join(training_path, 'Hispa')
leafblast_training_path = os.path.join(training_path, 'LeafBlast')

# Validation
brownspot_validation_path = os.path.join(validation_path, 'BrownSpot')
healthy_validation_path = os.path.join(validation_path, 'Healthy')
hispa_validation_path = os.path.join(validation_path, 'Hispa')
leafblast_validation_path = os.path.join(validation_path, 'LeafBlast')

In [14]:
# Create random list of items in validation data to be assigned as testing data
def split_data(validation_dir, testing_dir, split_size=0.5):
  data = os.listdir(validation_dir)
  shuffle = random.sample(data, len(data))
  n_test = math.ceil(len(shuffle) * split_size)

  test_list = shuffle[n_test:]
  val_list = shuffle[:n_test]

  for filename in test_list:
    file_path = os.path.join(validation_dir, filename)
    copy_path = os.path.join(testing_dir, filename)
    if os.path.getsize(file_path) != 0:
      os.rename(file_path, copy_path)

#split_data(brownspot_validation_path, brownspot_testing_path)
#split_data(healthy_validation_path, healthy_testing_path)
#split_data(hispa_validation_path, hispa_testing_path)
#split_data(leafblast_validation_path, leafblast_testing_path)

In [16]:
# Number of training, validation, and testing
print(f'Number of BrownSpot data: \n Training data: {len(os.listdir(brownspot_training_path))} \n Validation data: {len(os.listdir(brownspot_validation_path))} \n Testing data: {len(os.listdir(brownspot_testing_path))} \n')
print(f'Number of Healthy data: \n Training data: {len(os.listdir(healthy_training_path))} \n Validation data: {len(os.listdir(healthy_validation_path))} \n Testing data: {len(os.listdir(healthy_testing_path))} \n')
print(f'Number of Hispa data: \n Training data: {len(os.listdir(hispa_training_path))} \n Validation data: {len(os.listdir(hispa_validation_path))} \n Testing data: {len(os.listdir(hispa_testing_path))} \n')
print(f'Number of LeafBlast data: \n Training data: {len(os.listdir(leafblast_training_path))} \n Validation data: {len(os.listdir(leafblast_validation_path))} \n Testing data: {len(os.listdir(leafblast_testing_path))} \n')

Number of BrownSpot data: 
 Training data: 400 
 Validation data: 62 
 Testing data: 61 

Number of Healthy data: 
 Training data: 400 
 Validation data: 62 
 Testing data: 61 

Number of Hispa data: 
 Training data: 400 
 Validation data: 62 
 Testing data: 61 

Number of LeafBlast data: 
 Training data: 400 
 Validation data: 62 
 Testing data: 61 



In [41]:
img_size = (224, 224)

training_datagen = ImageDataGenerator(rescale=1/255,
                                      rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale=1/255)
testing_datagen = ImageDataGenerator(rescale=1/255)

training_dataset = training_datagen.flow_from_directory(training_path,
                                                          shuffle=True,
                                                          batch_size=32,
                                                          target_size=img_size)
validation_dataset = validation_datagen.flow_from_directory(validation_path,
                                                            shuffle=True,
                                                            batch_size=16,
                                                            target_size=img_size)
testing_dataset = testing_datagen.flow_from_directory(testing_path,
                                                      shuffle=True,
                                                      batch_size=16,
                                                      target_size=img_size)

Found 1600 images belonging to 4 classes.
Found 248 images belonging to 4 classes.
Found 244 images belonging to 4 classes.


# Model

In [77]:
img_shape = img_size + (3,)
base_model = tf.keras.applications.InceptionV3(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')

In [78]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_564 (Conv2D)            (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_575 (Batch  (None, 111, 111, 32  96         ['conv2d_564[0][0]']             
 Normalization)                 )                                                      

In [79]:
for layer in base_model.layers:
  if layer.name == 'batch_normalization_667':
    break
  layer.trainable = False
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_564 (Conv2D)            (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_575 (Batch  (None, 111, 111, 32  96         ['conv2d_564[0][0]']             
 Normalization)                 )                                                      

In [93]:
def custom_model_7(last_layer = base_model.output):
  x = tf.keras.layers.Flatten()(last_layer)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(4, activation='softmax')(x)

  model = tf.keras.Model(inputs=base_model.input, outputs=x)

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [94]:
custom_model_7 = custom_model_7()
custom_model_7.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_564 (Conv2D)            (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_575 (Batch  (None, 111, 111, 32  96         ['conv2d_564[0][0]']             
 Normalization)                 )                                                          

In [96]:
ckp_path = '/Users/bened/Python/Farmku/Rice/my_best_model_7.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(ckp_path, monitor='accuracy', verbose=2,
    save_best_only=True, mode='auto', save_freq='epoch')

In [97]:
history_7 = custom_model_7.fit(training_dataset,
                               validation_data = validation_dataset,
                               epochs = 200,
                               verbose = 2,
                               callbacks=[checkpoint])

Epoch 1/200

Epoch 1: accuracy improved from -inf to 0.41563, saving model to /Users/bened/Python/Farmku/Rice\my_best_model_7.hdf5
50/50 - 187s - loss: 1.6098 - accuracy: 0.4156 - val_loss: 1.8195 - val_accuracy: 0.4476 - 187s/epoch - 4s/step
Epoch 2/200

Epoch 2: accuracy improved from 0.41563 to 0.48250, saving model to /Users/bened/Python/Farmku/Rice\my_best_model_7.hdf5
50/50 - 185s - loss: 1.2627 - accuracy: 0.4825 - val_loss: 1.4839 - val_accuracy: 0.4879 - 185s/epoch - 4s/step
Epoch 3/200

Epoch 3: accuracy improved from 0.48250 to 0.51000, saving model to /Users/bened/Python/Farmku/Rice\my_best_model_7.hdf5
50/50 - 184s - loss: 1.2205 - accuracy: 0.5100 - val_loss: 1.3766 - val_accuracy: 0.5000 - 184s/epoch - 4s/step
Epoch 4/200

Epoch 4: accuracy improved from 0.51000 to 0.52125, saving model to /Users/bened/Python/Farmku/Rice\my_best_model_7.hdf5
50/50 - 186s - loss: 1.1290 - accuracy: 0.5213 - val_loss: 1.2381 - val_accuracy: 0.5000 - 186s/epoch - 4s/step
Epoch 5/200

Epoch 

In [98]:
custom_model_7.save('/Users/bened/Python/Farmku/Rice/latest_model.hdf5')